In [1]:
from mlflow.tracking import MlflowClient
MLFLOW_TRACKING_URI = "sqlite:///mlflow.db"
import mlflow

#### Interacting with the MLflow tracking server
    
    The MlflowClient object allows us to interact with..,
    - an MLflow Tracking Server that creates and manages experiments and runs
    - an MLflow Registry Server that creates and manages registered models and model versions. 
    
    To instantiate it we need to pass a tracking URI and/or a registry URI

In [ ]:
client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)
client.create_experiment(name="test-experiment")


In [11]:
from mlflow.entities import ViewType

runs = client.search_runs(
    experiment_ids='1',
    filter_string="metrics.rmse < 6.7",
    run_view_type=ViewType.ACTIVE_ONLY,
    max_results=7,
    order_by=["metrics.rmse ASC"]
)

In [12]:
for run in runs:
    print(f"run id: {run.info.run_id}, rmse: {run.data.metrics['rmse']:.4f}")

run id: 0be489a24de845e48aa33f05fe63ffb3, rmse: 5.6950
run id: 4a740e144be74788b99eaf8d8c478c6b, rmse: 5.8430
run id: 4c2665a0d9c9417cb11e584f2f929b9c, rmse: 6.2007


In [14]:
import mlflow

mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)

In [15]:
run_id = "4a740e144be74788b99eaf8d8c478c6b"
model_uri = f"runs:/{run_id}/model"
mlflow.register_model(model_uri=model_uri, name="nyc-taxi-regressor")

Registered model 'nyc-taxi-regressor' already exists. Creating a new version of this model...
2023/01/16 15:48:26 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: nyc-taxi-regressor, version 8
Created version '8' of model 'nyc-taxi-regressor'.


<ModelVersion: creation_timestamp=1673873306662, current_stage='None', description=None, last_updated_timestamp=1673873306662, name='nyc-taxi-regressor', run_id='4a740e144be74788b99eaf8d8c478c6b', run_link=None, source='./mlruns/1/4a740e144be74788b99eaf8d8c478c6b/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=8>

In [16]:
model_name = "nyc-taxi-regressor"
latest_versions = client.get_latest_versions(name=model_name)

for version in latest_versions:
    print(f"version: {version.version}, stage: {version.current_stage}")

version: 8, stage: None
version: 7, stage: Staging


In [17]:
model_version = 8
new_stage = "Staging"
client.transition_model_version_stage(
    name=model_name,
    version=model_version,
    stage=new_stage,
    archive_existing_versions=False
)

<ModelVersion: creation_timestamp=1673873306662, current_stage='Staging', description=None, last_updated_timestamp=1673873381569, name='nyc-taxi-regressor', run_id='4a740e144be74788b99eaf8d8c478c6b', run_link=None, source='./mlruns/1/4a740e144be74788b99eaf8d8c478c6b/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=8>

In [18]:
from datetime import datetime

date = datetime.today().date()
client.update_model_version(
    name=model_name,
    version=model_version,
    description=f"The model version {model_version} was transitioned to {new_stage} on {date}"
)

<ModelVersion: creation_timestamp=1673873306662, current_stage='Staging', description='The model version 8 was transitioned to Staging on 2023-01-16', last_updated_timestamp=1673873488982, name='nyc-taxi-regressor', run_id='4a740e144be74788b99eaf8d8c478c6b', run_link=None, source='./mlruns/1/4a740e144be74788b99eaf8d8c478c6b/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=8>

### Comparing versions and selecting the new \"Production\" model
 
    In the last section, we will retrieve models registered in the model registry and compare their performance on an unseen test set. The idea is to simulate the scenario in which a deployment engineer has to interact with the model registry to decide whether to update the model version that is in production or not.\n",
    
    These are the steps:
    
    1. Load the test dataset, which corresponds to the NYC Green Taxi data from the month of March 2021.
    2. Download the `DictVectorizer` that was fitted using the training data and saved to MLflow as an artifact, and load it with pickle.
    3. Preprocess the test set using the `DictVectorizer` so we can properly feed the regressors.
    4. Make predictions on the test set using the model versions that are currently in the Staging and Production stages, and compare their performance
    5. Based on the results, update the Production model version accordingly.
    
    
    **Note: the model registry doesn't actually deploy the model to production when you transition a model to the Production stage, it just assign a label to that model version. You should complement the registry with some CI/CD code that does the actual deployment.**

In [19]:
from sklearn.metrics import mean_squared_error
import pandas as pd


def read_dataframe(filename):
    df = pd.read_parquet(filename)

    df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
    df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df


def preprocess(df, dv):
    df['PU_DO'] = df['PULocationID'] + df['DOLocationID']
    categorical = ['PU_DO']
    numerical = ['trip_distance']
    train_dicts = df[categorical + numerical].to_dict(orient='records')
    return dv.transform(train_dicts)


def test_model(name, stage, X_test, y_test):
    model = mlflow.pyfunc.load_model(f"models:/{name}/{stage}")
    y_pred = model.predict(X_test)
    return {"rmse": mean_squared_error(y_test, y_pred, squared=False)}

#### Download new dataset to validate model

In [20]:
#test on march data
file = './data/green_tripdata_2022-03.parquet'
df = read_dataframe('./data/green_tripdata_2022-03.parquet')

In [21]:
client.download_artifacts(run_id=run_id, path='preprocessor', dst_path='.')

/tmp/ipykernel_3357/2852634549.py:1: FutureWarning: ``mlflow.tracking.client.MlflowClient.download_artifacts`` is deprecated since 2.0. This method will be removed in a future release. Use ``mlflow.artifacts.download_artifacts`` instead.
  client.download_artifacts(run_id=run_id, path='preprocessor', dst_path='.')


'/home/forkbo/python/mlops-project/02_experiment_tracking/preprocessor'

In [22]:
import pickle

with open("preprocessor/preprocessor.b", "rb") as f_in:
    dv = pickle.load(f_in)

In [23]:
X_test = preprocess(df, dv)

In [24]:
target = "duration"
y_test = df[target].values

####  Test model on new dataset

In [25]:
%time test_model(name=model_name, stage="Production", X_test=X_test, y_test=y_test)

[15:58:58] WARNING: ../src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
CPU times: user 464 ms, sys: 28.5 ms, total: 492 ms
Wall time: 340 ms


{'rmse': 8.067235777962093}

In [26]:
time test_model(name=model_name, stage="Staging", X_test=X_test, y_test=y_test)

CPU times: user 186 ms, sys: 17.1 ms, total: 203 ms
Wall time: 237 ms


{'rmse': 6.301359977603854}

####  Transition model from one stage to another best on test result

In [ ]:
model_version = 8
new_stage = "Production"
client.transition_model_version_stage(
    name=model_name,
    version=model_version,
    stage=new_stage,
    archive_existing_versions=True
)